<h1 align='center'>Fake News Detection</h1>

<h2>Importing Libraries And Datasets</h2>

In [97]:
import nltk
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model

In [98]:
# Loading The Data
df_true = pd.read_csv("True.csv")
df_fake = pd.read_csv("Fake.csv")

<h2>Performing Exploratory Data Analysis</h2>

In [99]:
# Adding A Target Class Column To Indicate Whether The News Is Real Or Fake
df_true['isfake'] = 1

In [100]:
df_fake['isfake'] = 0

In [101]:
# Concatenating Real And Fake News
df = pd.concat([df_true, df_fake]).reset_index(drop = True)


<h2>Performing Data Cleaning</h2>

In [102]:
# Obtaining Additional Stopwords From nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('portuguese')


In [103]:
# Removing Stopwords And Remove Words With 2 Or Less Characters
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
            result.append(token)
            
    return result

In [104]:
# Applying The Function To The Dataframe
df['clean'] = df['text'].apply(preprocess)

In [105]:
# Obtaining The Total Words Present In The Dataset
list_of_words = []
for i in df.clean:
    for j in i:
        list_of_words.append(j)

In [106]:
len(list_of_words)

2388359

In [107]:
# Obtaining The Total Number Of Unique Words
total_words = len(list(set(list_of_words)))
total_words

76697

In [108]:
# Joining The Words Into A String
df['clean_joined'] = df['clean'].apply(lambda x: " ".join(x))

In [109]:
# Determining The Maximum Number Of Words In Any Document Required To Create Word Embeddings 
maxlen = -1
for doc in df.clean_joined:
    tokens = nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen = len(tokens)
print("The maximum number of words in any document is =", maxlen)

The maximum number of words in any document is = 3904


<h2>Preparing The Data By Performing Tokenization And Padding</h2>

In [148]:
# Splitting Data Into Test And Train 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.clean_joined, df.isfake, test_size = 0.2)

In [149]:
from nltk import word_tokenize

In [150]:
# Creating A Tokenizer To Tokenize The Words And Create Sequences Of Tokenized Words
tokenizer = Tokenizer(num_words = total_words)
tokenizer.fit_on_texts(x_train)
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)


In [151]:
# Adding Padding
padded_train = pad_sequences(train_sequences,maxlen = 40, padding = 'post', truncating = 'post')
padded_test = pad_sequences(test_sequences,maxlen = 40, truncating = 'post') 

<h2>Building And Training The Model</h2>

In [152]:
# Sequential Model
model = Sequential()

# Embeddidng layer
model.add(Embedding(total_words, output_dim = 128))


# Bi-Directional LSTM
model.add(Bidirectional(LSTM(128)))

# Dense layers
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1,activation= 'sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 128)         9817216   
                                                                 
 bidirectional_8 (Bidirectio  (None, 256)              263168    
 nal)                                                            
                                                                 
 dense_16 (Dense)            (None, 128)               32896     
                                                                 
 dense_17 (Dense)            (None, 1)                 129       
                                                                 
Total params: 10,113,409
Trainable params: 10,113,409
Non-trainable params: 0
_________________________________________________________________


In [153]:
y_train = np.asarray(y_train)

In [154]:
# Training the model
model.fit(padded_train, y_train, validation_split = 0.1, epochs = 5)

Epoch 1/5
162/162 [==============================] - 7s 27ms/step - loss: 0.4249 - acc: 0.7941 - val_loss: 0.3281 - val_acc: 0.8698
Epoch 2/5
162/162 [==============================] - 4s 24ms/step - loss: 0.0765 - acc: 0.9766 - val_loss: 0.3638 - val_acc: 0.8715
Epoch 3/5
162/162 [==============================] - 4s 24ms/step - loss: 0.0138 - acc: 0.9967 - val_loss: 0.6363 - val_acc: 0.8576
Epoch 4/5
162/162 [==============================] - 4s 24ms/step - loss: 0.0077 - acc: 0.9985 - val_loss: 0.5270 - val_acc: 0.8524
Epoch 5/5
162/162 [==============================] - 4s 23ms/step - loss: 6.1749e-04 - acc: 0.9998 - val_loss: 0.7872 - val_acc: 0.8576


<h2>Assessing Trained Model Performance


In [163]:
# Making prediction
pred = model.predict(padded_test)

In [164]:
# If The Predicted Value Is >0.95 (i.e., More Than 95%), It Is Real Else It Is Fake
prediction = []
for i in range(len(pred)):
    if pred[i].item() > 0.95:
        prediction.append(1)
    else:
        prediction.append(0)

In [165]:
# Getting The Accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy = accuracy_score(list(y_test), prediction)
f1 = f1_score(list(y_test), prediction)

print("Model Accuracy : ", accuracy)
print("F1-Score : ", f1)

Model Accuracy :  0.8458333333333333
F1-Score :  0.8377192982456141


In [166]:
from sklearn.metrics import classification_report
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85       726
           1       0.88      0.80      0.84       714

    accuracy                           0.85      1440
   macro avg       0.85      0.85      0.85      1440
weighted avg       0.85      0.85      0.85      1440

